In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import data

In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np

In [9]:
# Instantiates the device to be used as GPU/CPU based on availability
device_gpu = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:

df_train = pd.read_csv("/kaggle/input/aksharantar/aksharantar_sampled/mai/mai_train.csv")
df_test = pd.read_csv('/kaggle/input/aksharantar/aksharantar_sampled/mai/mai_test.csv')
df_valid = pd.read_csv('/kaggle/input/aksharantar/aksharantar_sampled/mar/mar_valid.csv')

In [11]:
print(df_train)
print(df_test)
print(df_valid)

               chainij       चाइनिज
0            deyuralee      देउराली
1             dekhaint       देखैंत
2          tarkariwali   तरकारीवाली
3          kathapathme     कथापाठमे
4             dehalvee       देहलवी
...                ...          ...
51194      mahaamnaaka      महामनाक
51195           nandao        नन्दओ
51196  upbhoktaasabhak  उपभोक्तासभक
51197        kirauntee      किराँती
51198     ajeetnarayan   अजीतनारायण

[51199 rows x 2 columns]
         kaaranean      कारणेँ
0           odisha       ओडिशा
1         pembroke    पेमब्रोक
2           baxter     बैक्सटर
3        nizamabad   निजामाबाद
4         umedwaar     उमेदवार
...            ...         ...
4090  ganganarayan  गंगानारायण
4091      longview    लोंगव्यू
4092        andhra       आंध्र
4093          mays         मेज
4094     frederick    फ्रेडरिक

[4095 rows x 2 columns]
      garvyabarobarach  गारव्याबरोबरच
0                  reo           रियो
1         sangrahalaye     संग्रहालये
2                sidni         

In [12]:
eng_alphabets = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
pad_char = '-PAD-'

eng_alpha2index = {pad_char:0}
for index, alpha in enumerate(eng_alphabets):
  eng_alpha2index[alpha] = index+1

print(eng_alpha2index)

{'-PAD-': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26}


In [13]:
hindi_alphabets = [chr(alpha) for alpha in range(2304, 2432)]
hindi_alphabet_size = len(hindi_alphabets)

hindi_alpha2index = {pad_char : 0}
for index, alpha in enumerate(hindi_alphabets):
  hindi_alpha2index[alpha] = index + 1

print(hindi_alpha2index)
print(len(hindi_alpha2index))


{'-PAD-': 0, 'ऀ': 1, 'ँ': 2, 'ं': 3, 'ः': 4, 'ऄ': 5, 'अ': 6, 'आ': 7, 'इ': 8, 'ई': 9, 'उ': 10, 'ऊ': 11, 'ऋ': 12, 'ऌ': 13, 'ऍ': 14, 'ऎ': 15, 'ए': 16, 'ऐ': 17, 'ऑ': 18, 'ऒ': 19, 'ओ': 20, 'औ': 21, 'क': 22, 'ख': 23, 'ग': 24, 'घ': 25, 'ङ': 26, 'च': 27, 'छ': 28, 'ज': 29, 'झ': 30, 'ञ': 31, 'ट': 32, 'ठ': 33, 'ड': 34, 'ढ': 35, 'ण': 36, 'त': 37, 'थ': 38, 'द': 39, 'ध': 40, 'न': 41, 'ऩ': 42, 'प': 43, 'फ': 44, 'ब': 45, 'भ': 46, 'म': 47, 'य': 48, 'र': 49, 'ऱ': 50, 'ल': 51, 'ळ': 52, 'ऴ': 53, 'व': 54, 'श': 55, 'ष': 56, 'स': 57, 'ह': 58, 'ऺ': 59, 'ऻ': 60, '़': 61, 'ऽ': 62, 'ा': 63, 'ि': 64, 'ी': 65, 'ु': 66, 'ू': 67, 'ृ': 68, 'ॄ': 69, 'ॅ': 70, 'ॆ': 71, 'े': 72, 'ै': 73, 'ॉ': 74, 'ॊ': 75, 'ो': 76, 'ौ': 77, '्': 78, 'ॎ': 79, 'ॏ': 80, 'ॐ': 81, '॑': 82, '॒': 83, '॓': 84, '॔': 85, 'ॕ': 86, 'ॖ': 87, 'ॗ': 88, 'क़': 89, 'ख़': 90, 'ग़': 91, 'ज़': 92, 'ड़': 93, 'ढ़': 94, 'फ़': 95, 'य़': 96, 'ॠ': 97, 'ॡ': 98, 'ॢ': 99, 'ॣ': 100, '।': 101, '॥': 102, '०': 103, '१': 104, '२': 105, '३': 106, '४': 107, '५': 108, '६': 109, '७': 

In [14]:
torch.tensor([hindi_alpha2index[i] for i in list(df_train.iloc[0][1])])

tensor([39, 72, 10, 49, 63, 51, 65])

# Parameter Declaration

In [15]:
INPUT_DIM = len(eng_alpha2index)
OUTPUT_DIM = len(hindi_alpha2index)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5